# Image registration metrics for the unstain2stain images:
-Mutual Information with both stain and unstain in grayscale
-Target Registration Error (TRE)


In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot as plt
from glob import glob
from skimage.registration import optical_flow_tvl1
from time import time
from skimage.transform import warp
import matplotlib
import seaborn as sns

### Mutual Information (MI):

In [2]:
# from skin_2D_workflow\image_registration_evaluation to calculate mutual information:
# Function to calculate MI:
def calculate_mutual_information(img_path_1,img_path_2): #order doesn't matter, MI(A,B) = MI(B,A)
    img1 = np.array(cv2.imread(img_path_1))
    img2 = np.array(cv2.imread(img_path_2))
    img1_g = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2_g = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1)  # marginal x over y
    py = np.sum(pxy, axis=0)  # marginal y over x
    px_py = px[:, None] * py[None, :]  # broadcast to multiply marginals
    # now we can do the calculation using the pxy, px_py 2D arrays
    nonzeros = pxy > 0  # filer out the zero values
    mi = np.sum(pxy[nonzeros] * np.log(pxy[nonzeros] / px_py[nonzeros]))
    return mi

In [14]:
# img_path_1 = glob(os.path.join(r"\\shelter\Kyu\unstain2stain\biomax_images\registrated_images","**","**.png"),recursive=True) #unstained, 174 images same order
# img_path_2 = glob(os.path.join(r"\\shelter\Kyu\unstain2stain\biomax_images\stained\padded_images","**","**.png"),recursive=True) #stained, 174 images same order
img_path_1 = glob(os.path.join(r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he',"**.png"),recursive=True) #unstained, 503 images same order
img_path_2 = glob(os.path.join(r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\Unstained\OTS_14684_3',"**.png"),recursive=True) #stained, 503 images same order

### TODO: Create excel file or MI values for all files:

In [16]:
# MI for each pair of image:
mi_ra = []
for idx in range(0,len(img_path_1)):
    img1 = img_path_1[idx]
    img2 = img_path_2[idx]
    mi = calculate_mutual_information(img1,img2)
    mi_ra.append(mi)
print(mi_ra)

[0.3680207347756098, 0.3863792026750146, 0.3486229989674943, 0.37796195349547473, 0.3429490276289727, 0.2699525795011519, 0.4223023494711829, 0.45293873120050976, 0.3976530587703127, 0.41791243289626223, 0.4200109103352637, 0.3778279273195168, 0.24320567778165142, 0.3578063771107999, 0.3731781219960698, 0.1707067987821535, 0.3515023576136908, 0.3327430896986511, 0.3145152184936518, 0.17829054558224705, 0.1658965468796515, 0.1943594932000115, 0.3782347103099781, 0.3067898624312263, 0.281439193703071, 0.267017531662309, 0.22176236836769636, 0.29140093952410934, 0.2559785037700864, 0.3568955044147358, 0.2861107879969284, 0.29323395397767127, 0.4135507816531908, 0.3388320017086665, 0.29284963460748836, 0.2956719514676222, 0.2946161291534203, 0.2888518619992587, 0.25891847928713424, 0.3463085486003298, 0.3951622988993406, 0.4393129844255049, 0.301873726554128, 0.33296994101435795, 0.32442092382468124, 0.3451069867390978, 0.33143081218950676, 0.19389571768274616, 0.31461768287435854, 0.43546

In [5]:
print(min(mi_ra))
print(np.argmin(mi_ra))

0.020379276123480325
297


In [6]:
print(max(mi_ra))
print(np.argmax(mi_ra))

0.48377640474942707
67


### Look at pictures with minimum and maximum values in grayscale and RGB, compare in powerpoint!


In [37]:
# minimum value:
img1 = np.array(cv2.imread(img_path_1[np.argmin(mi_ra)]))
img2 = np.array(cv2.imread(img_path_2[np.argmin(mi_ra)]))
img1_g = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2_g = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
print(img_path_1[np.argmin(mi_ra)])
print(img_path_2[np.argmin(mi_ra)])

\\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy3273.png
\\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\Unstained\OTS_14684_3\xy3273.png


In [36]:
# maximum value:
img1 = np.array(cv2.imread(img_path_1[np.argmax(mi_ra)]))
img2 = np.array(cv2.imread(img_path_2[np.argmax(mi_ra)]))
img1_g = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2_g = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
print(img_path_1[np.argmax(mi_ra)])
print(img_path_2[np.argmax(mi_ra)])

max path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy1782.png
min path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\Unstained\OTS_14684_3\xy1782.png


### Seems like for min, it's images with oil gland/hair shaft. Seems like for max, it's images with red blood cells and fat. TODO: Maybe check top 5 images instead of doing just one image for max/min to see?

In [35]:
sorted_list = np.argsort(mi_ra)
top_five_min_idx = sorted_list[0:5] # 0th is lowest
top_five_max_idx = sorted_list[-5:] # 4th is highest
for idx in range(0,5):
    min_idx = mi_ra.index([mi_ra[top_five_min_idx[idx]]])
    max_idx = mi_ra.index([mi_ra[top_five_max_idx[idx]]])
    print("min img path",img_path_1[min_idx])
    print("max img path",img_path_1[max_idx])

min img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy3273.png
max img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy1274.png
min img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy3274.png
max img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy1187.png
min img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy2895.png
max img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy0751.png
min img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy3276.png
max img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy1656.png
min img path \\shelter\Kyu\unstain2stain\unstain2stain_tile\cyclegan(matlab)\train\HE\OTS_14684_3_he\xy3

### Then do a mi_ra analysis of the photo with min and max mi value with all the other photos, compare MI to see the magnitude difference (if MI can be trustworthy!)

In [19]:
def calculate_mutual_information_v2(img_path_src,img_list_path): #order doesn't matter, MI(A,B) = MI(B,A)
    img1 = np.array(cv2.imread(img_path_src)) # find one image
    img1_g = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    mi_ra = [];
    for idx in range(0,len(img_list_path)):
        img2 = np.array(cv2.imread(img_list_path[idx])) # all the other images
        img2_g = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
        pxy = hist2d / float(np.sum(hist2d))
        px = np.sum(pxy, axis=1)  # marginal x over y
        py = np.sum(pxy, axis=0)  # marginal y over x
        px_py = px[:, None] * py[None, :]  # broadcast to multiply marginals
        # now we can do the calculation using the pxy, px_py 2D arrays
        nonzeros = pxy > 0  # filer out the zero values
        mi = np.sum(pxy[nonzeros] * np.log(pxy[nonzeros] / px_py[nonzeros]))
        mi_ra.append(mi)
    return mi_ra

In [26]:
# min:
mi_list_1 = calculate_mutual_information_v2(img_path_1[np.argmin(mi_ra)],img_path_2)
print(img_path_1[np.argmax(mi_list_1)]) # max since we expect it still to be max

\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\HE\OTS_14684_3_he\xy3273.png


In [27]:
print(len(mi_list_1))
del(mi_list_1[np.argmax(mi_list_1)])
print(len(mi_list_1)) #deleted
print("mean is",np.mean(mi_list_1))

503
502
mean is 0.0012111883689702536


In [28]:
np.max(mi_list_1)

0.010166226985892254

### Recall that the matching image had a MI value of 0.02 (minimum). But for this image, MI amongst other has an average of 0.0012, with the max value being 0.01, so half of 0.02. So the average magnitude difference is about 10 fold. Let's see this for the max as well:

In [25]:
# max:
mi_list_2 = calculate_mutual_information_v2(img_path_1[np.argmax(mi_ra)],img_path_2)
print(img_path_1[np.argmax(mi_list_2)])

\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\HE\OTS_14684_3_he\xy1782.png


In [29]:
print(len(mi_list_2))
del(mi_list_2[np.argmax(mi_list_2)])
print(len(mi_list_2)) #deleted
print("mean is",np.mean(mi_list_2))

503
502
mean is 0.00267233332965388


In [30]:
np.max(mi_list_2)

0.02300343978090489

## For this, the difference is bigger. Recall our value for MI was 0.48. The average is 0.0026, and the max is 0.023. Magnitude difference is huge. Seems like MI is a trustworthy enough metric for this case! TODO: Maybe draw a histogram to see the distributed values?

### Now try to see how MI changes with pixel offset from -10 to 10. Crop the photo by x amount (x>10) and offset NS and HE, see difference in MI. Plot the difference.

In [4]:
def calculate_mutual_information_v3(img1ra,img2ra): #order doesn't matter, MI(A,B) = MI(B,A)
    img1_g = cv2.cvtColor(img1ra,cv2.COLOR_BGR2GRAY)
    img2_g = cv2.cvtColor(img2ra,cv2.COLOR_BGR2GRAY)
    hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1)  # marginal x over y
    py = np.sum(pxy, axis=0)  # marginal y over x
    px_py = px[:, None] * py[None, :]  # broadcast to multiply marginals
    # now we can do the calculation using the pxy, px_py 2D arrays
    nonzeros = pxy > 0  # filer out the zero values
    mi = np.sum(pxy[nonzeros] * np.log(pxy[nonzeros] / px_py[nonzeros]))
    return mi

In [5]:
# Try it with the minimum image
hesrc = r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\HE\OTS_14684_3_he\xy1782.png'
nssrc = r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\Unstained\OTS_14684_3\xy1782.png'
img1 = Image.open(hesrc)
img2 = Image.open(nssrc)
img1ra = np.array(img1)
img2ra = np.array(img2)
# First crop both img1 and img2 by 20 pixels.
x1 = 50
x2 = 974
croppedhe = img1.crop(box = (x1,x1,x2,x2)) #924 x 924
croppedhe = np.array(croppedhe)
croppedns = img2.crop(box = (x1,x1,x2,x2)) #924 x 924

In [12]:
# Try mismatching the he with normal image ns, with positive offset
all_mi_list = [];
for idx in range(0, 11):  # 0~10
    mismatch_crop_he = img1.crop(box=(x1 + idx, x1, x2 + idx, x2))
    mismatch_crop_he = np.array(mismatch_crop_he)
    normal_ns = np.array(croppedns)
    tmp_mi = calculate_mutual_information_v3(mismatch_crop_he, normal_ns)
    all_mi_list.append(tmp_mi)
del(all_mi_list[0])
all_mi_list = all_mi_list
all_mi_list

[0.47427731636736725,
 0.38951439662347964,
 0.32219628559239194,
 0.27583064302537247,
 0.2458637059512731,
 0.22695703438466186,
 0.2148303701398396,
 0.20630027790217947,
 0.1995937216795915,
 0.19440811713508332]

In [13]:
all_mi_list1 = [];
for idx in range(0,11): # 0~10
    mismatch_crop_he = img1.crop(box = (x1-idx,x1, x2-idx,x2))
    mismatch_crop_he = np.array(mismatch_crop_he)
    normal_ns = np.array(croppedns)
    tmp_mi1 = calculate_mutual_information_v3(mismatch_crop_he,normal_ns)
    all_mi_list1.append(tmp_mi1)
all_mi_list1 = all_mi_list1[::-1]
all_mi_list1

[0.20122516764271367,
 0.2064838752486673,
 0.2125963624160035,
 0.22099605974191183,
 0.23295582478523752,
 0.25275703405606803,
 0.28848965741586013,
 0.3474604183598314,
 0.43023631334396906,
 0.515301980860861,
 0.5391774632371988]

In [14]:
complete_mi_list = all_mi_list1 + all_mi_list
x = complete_mi_list
y = list(range(-10,11))

In [15]:
complete_mi_list

[0.20122516764271367,
 0.2064838752486673,
 0.2125963624160035,
 0.22099605974191183,
 0.23295582478523752,
 0.25275703405606803,
 0.28848965741586013,
 0.3474604183598314,
 0.43023631334396906,
 0.515301980860861,
 0.5391774632371988,
 0.47427731636736725,
 0.38951439662347964,
 0.32219628559239194,
 0.27583064302537247,
 0.2458637059512731,
 0.22695703438466186,
 0.2148303701398396,
 0.20630027790217947,
 0.1995937216795915,
 0.19440811713508332]

In [16]:
pd.DataFrame({'x':x,'y':y}).to_csv(r'C:\Users\Kevin\Desktop\tmp3.csv')

### Use MI to compare between real,raw HE tiles and inferred HE tiles and compare the two, generate heatmap:

In [2]:
import os
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from natsort import natsorted

def calculate_mutual_information_v3(img1ra,img2ra):
    img1_g = cv2.cvtColor(img1ra,cv2.COLOR_BGR2GRAY)
    img2_g = cv2.cvtColor(img2ra,cv2.COLOR_BGR2GRAY)
    hist2d, x_edges, y_edges = np.histogram2d(img1_g.ravel(),img2_g.ravel(),bins=20)
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1)  # marginal x over y
    py = np.sum(pxy, axis=0)  # marginal y over x
    px_py = px[:, None] * py[None, :]  # broadcast to multiply marginals
    # now we can do the calculation using the pxy, px_py 2D arrays
    nonzeros = pxy > 0  # filer out the zero values
    mi = np.sum(pxy[nonzeros] * np.log(pxy[nonzeros] / px_py[nonzeros]))
    return mi

In [ ]:
inferred_he_tiles_src = r'\\shelter\Kyu\unstain2stain\generated_fake_images\inferred_tiles\fake_OTS_14832_5he'
real_he_tiles_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE\OTS_14832_5_he'

# rows and cols from the return_row_col function in predict_v1.ipynb
heatmap_rows = 115
heatmap_cols = 86
heatmap = np.zeros(shape=((heatmap_rows), (heatmap_cols)))
inferred_files = natsorted(os.listdir(inferred_he_tiles_src))
real_files = natsorted(os.listdir(real_he_tiles_src))
inferred_files = [x for x in inferred_files if x.endswith(".png")]
print("inferred_tiles len before",len(inferred_files))
del(inferred_files[-1])
real_files = [x for x in real_files if x.endswith(".png")]
print("inferreid tiles afteR", len(inferred_files))
print("real file len after",len(real_files))
for i in tqdm(range(len(real_files)),desc="Images calculated",colour='red'):
    inferred_file_path = os.path.join(inferred_he_tiles_src, inferred_files[i])
    real_file_path = os.path.join(real_he_tiles_src, real_files[i])

    inferred_image = cv2.imread(inferred_file_path)
    real_image = cv2.imread(real_file_path)

    mi_score = calculate_mutual_information_v3(inferred_image, real_image)
    print(mi_score,inferred_files[i],real_files[i])
    y , x = map(int, real_files[i].split('xy')[0].split('.png')[0].split('_'))
    x = (x - 1126)/1024
    y = (y - 1126)/1024
    np.put(heatmap, y + x, mi_score)
    # if x >= 0 and x < heatmap.shape[1] and y >= 0 and y < heatmap.shape[0]:
    #     heatmap[y,x] = mi_score

fig = plt.figure(figsize=(20, 20))
sns.heatmap(heatmap, cmap='viridis', xticklabels=False, yticklabels=False)
plt.axis('off')
save_src = r'\\shelter\Kyu\unstain2stain\generated_fake_images\MI_heatmap_real_vs_infer'
plt.savefig(save_src, dpi=300, bbox_inches='tight')
plt.close(fig)


In [ ]:
heatmap

### Now, the code to generate the FID score can just be done using pytorch-fid in the command prompt, but I want to generate imagtes with gaussian noise and gaussian blur to see how it increases the original inferred image

In [5]:
# create gaussian noise: (from: https://www.kaggle.com/code/chanduanilkumar/adding-and-removing-image-noise-in-python/notebook)

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm

def apply_gaussian_noise(image_src, save_dir):
    gauss_noise=np.zeros((1024,1024,3),dtype=np.uint8)
    cv2.randn(gauss_noise,128,20)
    gauss_noise=(gauss_noise*0.5).astype(np.uint8)
    img = np.array(Image.open(image_src))
    gn_img=cv2.add(img,gauss_noise)
    Image.fromarray(gn_img).save(save_dir)

# define the source and target directories
src_dir = r'\\shelter\Kyu\unstain2stain\generated_fake_images\inferred_tiles'
target_dir = r'\\shelter\Kyu\unstain2stain\generated_fake_images\noised_tiles'

# iterate over all directories and files in the source directory
for root, dirs, files in os.walk(src_dir):
    # create the corresponding directories in the target directory
    target_root = root.replace(src_dir, target_dir)
    os.makedirs(target_root, exist_ok=True)

    # iterate over all files in the current directory
    for file in tqdm(files,desc="Files processed",colour='red'):
        # check if the file is a PNG image
        if file.lower().endswith('.png'):
            # construct the paths to the source and target files
            src_path = os.path.join(root, file)
            target_path = os.path.join(target_root, file)

            # apply the gaussian noise and save the image
            apply_gaussian_noise(src_path, target_path)


Files processed: 0it [00:00, ?it/s]
Files processed:  20%|█▉        | 2163/10996 [25:31<1:41:13,  1.45it/s]

### Now try Target Registration Error (TRE):
### paper using tre and optical flow: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9099354/

In [2]:
def registrate_two_images(reference_image_path, moving_image_path, save_path):
    """
    Note: The order of the files saved in the ref_img_path and mov_img_path must be the same so that you are registering the same images!
    """
    ref_img_path = [_ for _ in os.listdir(reference_image_path) if _.endswith(".png")]
    ref_img_path_complete = [os.path.join(reference_image_path, x) for x in ref_img_path]
    mov_img_path = [_ for _ in os.listdir(moving_image_path) if _.endswith(".png")]
    mov_img_path_complete = [os.path.join(moving_image_path, x) for x in mov_img_path]
    mov_img_name = [x.replace('.png','') for x in mov_img_path]
    if int(len(ref_img_path)) != int(len(mov_img_path)):
        print("Number of images in reference and moving file paths are not equal, please fix and try again!")
        return

    start = time()
    for idx in range(0,len(ref_img_path_complete)):
        ref_img = Image.open(ref_img_path_complete[idx])
        mov_img = Image.open(mov_img_path_complete[idx])
        ref_img = np.array(ref_img)
        mov_img = np.array(mov_img)
        ref_img_g = cv2.cvtColor(ref_img,cv2.COLOR_RGBA2GRAY)
        mov_img_g = cv2.cvtColor(mov_img,cv2.COLOR_RGBA2GRAY)
        v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
        print("v",v.shape)
        print("u",u.shape)
        nr, nc = ref_img_g.shape
        row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                             indexing='ij')

        mov_img_warp_ra =[]
        for i in range(3):
            #mov_img warped (2d array) = mov img (before warped) (2d array since each channel) warped by optical flow components v u of each axis (2 makes 2d), which is added to row_coords and col_coords of the ref img (fixed image).
            mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
            print(mov_img[:,:,i])
            print(mov_img[:,:,i].shape)
            print(np.array([row_coords + v, col_coords + u]))
            print(np.array([row_coords + v, col_coords + u]).shape)
            print(mov_img_warp)
            print(mov_img_warp.shape)
            mov_img_warp_ra.append(mov_img_warp)
        r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
        g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
        b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
        rgb = np.stack([r,g,b],axis=2)
        reg_img = Image.fromarray(rgb)
        print(idx)
        # reg_img.save(os.path.join(save_path,str(mov_img_name[idx]) + '.png'))

    end = time()
    print("time it took to register: "+  str((end-start)/60) + " minutes")


NameError: name 'apply_gaussian_noise' is not defined

In [40]:
registrate_two_images(reference_image_path=r'\\fatherserverdw\Kevin\imageregistration2\padded_images',moving_image_path=r'\\fatherserverdw\Kevin\imageregistration2\registered_images',save_path = None)

v (6328, 5352)
u (6328, 5352)
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 237 237 247]
 [255 255 255 ... 237 237 247]
 [255 255 255 ... 237 237 246]]
(6328, 5352)
[[[  93.27698517   93.27725983   93.2778244  ...   54.83100891
     54.83141327   54.83161163]
  [  94.27669525   94.27696228   94.27754211 ...   55.83059311
     55.83098602   55.83118439]
  [  95.27610016   95.27638245   95.27693939 ...   56.82975769
     56.83015442   56.83033752]
  ...
  [6393.63539124 6393.63536835 6393.63533783 ... 6397.95688629
   6397.95678711 6397.95672607]
  [6394.63547516 6394.6354599  6394.63542175 ... 6398.95691681
   6398.95679474 6398.95674133]
  [6395.63552094 6395.63549805 6395.63546753 ... 6399.95692444
   6399.95681    6399.95674896]]

 [[  54.94009399   55.94036102   56.94089508 ... 5369.51122856
   5370.51117706 5371.51115227]
  [  54.93984604   55.94010925   56.94064331 ... 5369.51142502
   5370.51137352 5371.5113487


KeyboardInterrupt



In [38]:
b = np.meshgrid(np.arange(6328),np.arange(5352), indexing='ij')
b[1].shape

(6328, 5352)

In [32]:
def target_registration_error(reference_image,moving_image, display_errors = False, min_err = None, max_err = None, figure_size=(8,6)):
    """
    Edited from: https://github.com/SimpleITK/SPIE2018_COURSE/blob/master/utilities.py
    """
    start = time()
    ref_img = Image.open(reference_image)
    mov_img = Image.open(moving_image)
    ref_img = np.array(ref_img)
    mov_img = np.array(mov_img)
    ref_img_g = cv2.cvtColor(ref_img,cv2.COLOR_RGBA2GRAY)
    mov_img_g = cv2.cvtColor(mov_img,cv2.COLOR_RGBA2GRAY)
    v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
    points_ra = #2d array of some points from fixed image (np.meshgrid)
    points_list = # list of points_ra
    transformed_points_ra = warp(points,np.array([v, u]),mode='edge')
    transformed_points_list =
      errors = [np.linalg.norm(np.array(p_fixed) -  np.array(p_moving)) # TRE = transformed point of fixed image - ref point of the moving image (the original point of fixed image and original point of moving image should be the "same" point of the image
            for p_fixed,p_moving in zip(transformed_point_list, reference_point_list)]
    if display_errors:
        fig = plt.figure(figsize=figure_size)
        ax = fig.add_subplot(111, projection='3d')
    if not min_err:
        min_err = np.min(errors)
    if not max_err:
        max_err = np.max(errors)

    collection = ax.scatter(list(np.array(point_list).T)[0],
                            list(np.array(point_list).T)[1],
                            list(np.array(point_list).T)[2],
                            marker = 'o',
                            c = errors,
                            vmin = min_err,
                            vmax = max_err,
                            cmap = matplotlib.cm.hot,
                            label = 'original points')
    plt.colorbar(collection, shrink=0.8)
    plt.title('registration errors in mm', x=0.7, y=1.05)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

return errors

SyntaxError: invalid syntax (2329407227.py, line 13)

In [3]:
def target_registration_errors(tx, point_list, reference_point_list,
                               display_errors = False, min_err= None, max_err=None, figure_size=(8,6)):
  """
  Distances between points transformed by the given transformation and their
  location in another coordinate system. When the points are only used to
  evaluate registration accuracy (not used in the registration) this is the
  Target Registration Error (TRE).
  Args:
      tx (SimpleITK.Transform): The transform we want to evaluate.
      point_list (list(tuple-like)): Points in fixed image
                                     coordinate system.
      reference_point_list (list(tuple-like)): Points in moving image
                                               cooredinate system.
      display_errors (boolean): Display a 3D figure with the points from
                                point_list color corresponding to the error.
      min_err, max_err (float): color range is linearly stretched between min_err
                                and max_err. If these values are not given then
                                the range of errors computed from the data is used.
      figure_size (tuple): Figure size in inches.
  Returns:
   (errors) [float]: list of TRE values.
  """
  transformed_point_list = [tx.TransformPoint(p) for p in point_list] # transform the fixed point from fixed image

  errors = [np.linalg.norm(np.array(p_fixed) -  np.array(p_moving)) # TRE = transformed point of fixed image - ref point of the moving image (the original point of fixed image and original point of moving image should be the "same" point of the image
            for p_fixed,p_moving in zip(transformed_point_list, reference_point_list)]
  if display_errors:
      from mpl_toolkits.mplot3d import Axes3D
      import matplotlib.pyplot as plt
      import matplotlib
      fig = plt.figure(figsize=figure_size)
      ax = fig.add_subplot(111, projection='3d')
      if not min_err:
          min_err = np.min(errors)
      if not max_err:
          max_err = np.max(errors)

      collection = ax.scatter(list(np.array(point_list).T)[0],
                              list(np.array(point_list).T)[1],
                              list(np.array(point_list).T)[2],
                              marker = 'o',
                              c = errors,
                              vmin = min_err,
                              vmax = max_err,
                              cmap = matplotlib.cm.hot,
                              label = 'original points')
      plt.colorbar(collection, shrink=0.8)
      plt.title('registration errors in mm', x=0.7, y=1.05)
      ax.set_xlabel('X')
      ax.set_ylabel('Y')
      ax.set_zlabel('Z')
      plt.show()

  return errors

In [50]:
# maybe new image registration method? maybe use vedo as suggested in one solution in this link for 3d registration: https://forum.image.sc/t/measuring-registration-accuracy/62651/6